In [1]:
using JuliaDB, IterableTables, Dates, Printf;

┌ Info: Precompiling JuliaDB [a93385a2-3734-596a-9a66-3cfbb77141e6]
└ @ Base loading.jl:1189
┌ Info: Precompiling IterableTables [1c8ee90f-4401-5389-894e-7a04a3dc0f4d]
└ @ Base loading.jl:1189


In [2]:
function makeCompDict(wcafile)
    compids = String[]
    compdates = Date[]
    lines = readlines(wcafile)
    for i = 2:length(lines) # skip header line
        line = lines[i]
        data = split(line, "\t")
        push!(compids, data[1])
        compdate = string(data[6], "-", data[7], "-", data[8])
        push!(compdates, Date(compdate, "y-m-d"))
    end
        compDict = Dict(zip(compids, compdates))
        sortedCompsArray = sort(collect(zip(values(compDict),keys(compDict))))
    
    return compDict, sortedCompsArray
end

makeCompDict (generic function with 1 method)

In [10]:
function genGlickoArray(comp, event, round)
    
    myresults = table(results[comp, event, round, :])
    mytable = table(select(myresults, (:personId, :pos)), pkey=:pos)
    
    output = []
    for i in 1:length(mytable), j in (i+1):length(mytable)
        o = [mytable[i].personId, mytable[j].personId, 1]
        o2 = [mytable[j].personId, mytable[i].personId, 0]
        push!(output, o)
        push!(output, o2)
    end
    
    return output
end

genGlickoArray (generic function with 1 method)

In [5]:
function genAllGlickoArrays(event)
    compDict, sortedCompsArray = makeCompDict("WCA_export_Competitions.tsv")
    allGlickoArrays = []
    lengthOfComps = length(sortedCompsArray)
    for i in 1:length(sortedCompsArray)
        index = sortedCompsArray[i][1]
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "0"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "1"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "2"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "3"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "b"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "c"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "d"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "e"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "f"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "g"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "h"))
        print("\r $i / $lengthOfComps Comps  ||  $index")
    end
    output = []
    for i in 1:length(allGlickoArrays)
        if length(allGlickoArrays[i]) != 0
            push!(output, allGlickoArrays[i])
        end
    end
    return output
end

genAllGlickoArrays (generic function with 1 method)

In [57]:
function calcRating(p1,p2,sc, localratingdict, localRDdict, ratingdict, RDdict, voldict)
    tau = 0.5
    g(phi) = (1 + (3*(phi^2)*2)/(2*(pi^2)))^(-1/2)
    E(mu1, mu2, phi2) = 1/(1 + exp(-g(phi2)*(mu1 - mu2)))
    f(x) = (exp(x)*(delta^2 - phi1^2 - v - exp(x)))/(2*(phi1^2 + v + exp(x))^2) - (x-a)/(tau^2)
    
    r1 = ratingdict[p1]
    r2 = localratingdict[p2]
    RD1 = RDdict[p1]
    RD2 = localRDdict[p2]
    sigma1 = voldict[p1]
    
    mu1 = (r1 - 1500)/173.7178
    phi1 = RD1/173.7178
    mu2 = (r2 - 1500)/173.7178
    phi2 = RD2/173.7178
    
    v = ((g(phi2)^2)*E(mu1, mu2, phi2)*(1 - E(mu1, mu2, phi2)))^-1
    delta = v*(g(phi2)*(sc - E(mu1, mu2, phi2)))
    epsilon = 0.000001
    
    A = a = log(sigma1^2)
    if delta^2 > phi1^2 + v
        B = log(delta^2 - phi1^2 - v)
    else
        k = 1
        while f(a - k*tau) < 0
            k += 1
        end
        B = a - k*tau 
    end
    fA = f(A)
    fB = f(B)
    while abs(B-A) > epsilon
        C = A + ((A - B)*fA)/(fB - fA)
        fC = f(C)
        if fC*fB < 0 
            A = B
            fA = fB
        else
            fA = fA/2
        end
        B = C
        fB = fC
    end
    
    sigma1prime = exp(A/2)
    phistar = sqrt(phi1^2 + sigma1prime^2)
    phi1prime = 1/sqrt(1/(phistar^2) + 1/v)
    mu1prime = mu1 + (phi1prime^2)*(g(phi2)*(sc - E(mu1, mu2, phi2)))
   
    r1prime  = 173.7178*mu1prime + 1500
    RD1prime = 173.7178*phi1prime
    
    return r1prime, RD1prime, sigma1prime
end

calcRating (generic function with 3 methods)

In [71]:
function generateRatings(data, RD, vol)
    playerids = []
    for i in 1:length(wca_persons)
        push!(playerids, wca_persons[i][1])
    end

    #reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, RD)
        push!(vols, vol)
    end

    ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));
    
    for i in 1:length(data)
        localratingdict = ratingdict
        localRDdict = RDdict
        ratingdict, RDdict, voldict = calcRatings(data[i], localratingdict, localRDdict, ratingdict, RDdict, voldict)
        print("\r$i / ", length(data))
    end
    
    RatingList = sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)
    
    return RatingList   
end    

generateRatings (generic function with 2 methods)

In [59]:
function calcRatings(data, localratingdict, localRDdict, ratingdict, RDdict, voldict)
    for i in 1:length(data)
        r1prime, RD1prime, sigma1prime = calcRating(data[i][1], data[i][2], data[i][3], localratingdict, localRDdict, ratingdict, RDdict, voldict)
        ratingdict[data[i][1]] = r1prime
        RDdict[data[i][1]] = RD1prime
        voldict[data[i][1]] = sigma1prime
    end
    return ratingdict, RDdict, voldict
end
    

calcRatings (generic function with 2 methods)

In [69]:
function printRatings(RatingList)
    htb = @sprintf "%4s %-30s \t %-7s %-15s" "Rank" "Name" "Rating" "Citizen of"
    println()
    println(htb)
    for i in 1:length(RatingList)
        id = RatingList[i][2]
        name = persons[id, 1].name
        if length(name) > 30
            name = name[1:30]*"..."
        end
        rating = RatingList[i][1]
        country = persons[id, 1].countryId
        tb = @sprintf "%4d %30s \t %6.2f %-15s" i rpad(name, 30) rating rpad(country, 15)
        println(tb)
    end
end

printRatings (generic function with 1 method)

In [3]:
results = loadndsparse("WCA_export_Results.tsv"; delim = '\t',
          indexcols = [:competitionId, :eventId, :roundTypeId, :personId],
          colparsers=Dict("personId"=>String,"eventId"=>String,"competitionId"=>String, "roundTypeId"=>String, "pos" =>Int))

4-d NDSparse with 1965270 values (13 field named tuples):
    Dimensions#  colname        type
────────────────────────
1  competitionId  String
2  eventId        String
3  roundTypeId    String
4  personId       String
    Values#   colname                type
─────────────────────────────────
5   pos                    Int64
6   best                   Int64
7   average                Int64
8   personName             String
9   personCountryId        String
10  formatId               String
11  value1                 Int64
12  value2                 Int64
13  value3                 Int64
14  value4                 Int64
15  value5                 Int64
16  regionalSingleRecord   String
17  regionalAverageRecord  String

In [6]:
path_persons = "WCA_export_Persons.tsv"
wca_persons = loadtable(path_persons; delim = '\t', quotechar = '\\',
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

persons = loadndsparse(path_persons; delim = '\t', quotechar = '\\', indexcols = [:id, :subid],
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [20]:
thing = genAllGlickoArrays("333")

 5566 / 5566 Comps  ||  2019-07-06

14928-element Array{Any,1}:
 Any[Any["1982THAI01", "1982RAZO01", 1], Any["1982RAZO01", "1982THAI01", 0], Any["1982THAI01", "1982LABA01", 1], Any["1982LABA01", "1982THAI01", 0], Any["1982THAI01", "1982PETR01", 1], Any["1982PETR01", "1982THAI01", 0], Any["1982THAI01", "1982UENO01", 1], Any["1982UENO01", "1982THAI01", 0], Any["1982THAI01", "1982JEAN01", 1], Any["1982JEAN01", "1982THAI01", 0]  …  Any["1982GALR01", "1982TENE01", 1], Any["1982TENE01", "1982GALR01", 0], Any["1982GALR01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982GALR01", 0], Any["1982SEBE01", "1982TENE01", 1], Any["1982TENE01", "1982SEBE01", 0], Any["1982SEBE01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982SEBE01", 0], Any["1982TENE01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982TENE01", 0]]
 Any[Any["2003KNIG01", "2003AKIM01", 1], Any["2003AKIM01", "2003KNIG01", 0], Any["2003KNIG01", "2003ALLE01", 1], Any["2003ALLE01", "2003KNIG01", 0], Any["2003KNIG01", "2003VAND01", 1], Any["2003VAND01", "2003KNIG01", 0], Any["2003KNIG01", "1982

In [67]:
RatingList = generateRatings(thing, 35, 0.006)

14928 / 14928

116190-element Array{Tuple{Any,Any},1}:
 (3520.9440405477544, "2012PARK03")
 (3456.539347748617, "2009ZEMD01") 
 (3357.244878789832, "2010WEYE01") 
 (3223.00587419787, "2010WANG68")  
 (3204.44641492145, "2013NAHM01")  
 (3194.8727484707197, "2011ETTE01")
 (3179.3412557520587, "2010WEYE02")
 (3154.1345170167115, "2012PONC02")
 (3116.895331505723, "2007VALK01") 
 (3060.4675615607034, "2013GERH01")
 (3057.624536477354, "2014XUZI01") 
 (3048.592691258791, "2012CARL03") 
 (3010.8566888404366, "2012BEAH01")
 ⋮                                 
 (815.6621053020974, "2012SIDO01") 
 (815.0576388470544, "2014TRAY01") 
 (810.7890614420199, "2018WANG97") 
 (808.4815059944215, "2016HUAN47") 
 (807.2009064542543, "2014GUIL01") 
 (806.9830704133698, "2016BERL01") 
 (805.2552060194757, "2016VIKS01") 
 (799.4244115999846, "2016SANC43") 
 (775.0975083612126, "2012GUIL01") 
 (731.3006430195829, "2013CIUK02") 
 (703.1908673816719, "2012RONK01") 
 (659.8325508236318, "2015CLAR13") 

In [73]:
sum = 0
for i in 1:length(RatingList)
    sum += RatingList[i][1]
end
println("mean: ", sum/length(RatingList))
println("median: ", RatingList[Int.(length(RatingList)/2)][1])
println("min: ", RatingList[length(RatingList)][1])
println("max: ", RatingList[1][1])

mean: 1456.8872021233763
median: 1433.2399560022309
min: 659.8325508236318
max: 3520.9440405477544


In [70]:
printRatings(RatingList)


Rank Name                           	 Rating  Citizen of     
   1 Max Park                       	 3520.94 USA            
   2 Feliks Zemdegs                 	 3456.54 Australia      
   3 Philipp Weyer                  	 3357.24 Germany        
   4 Bill Wang                      	 3223.01 Canada         
   5 Seung Hyuk Nahm (남승혁)          	 3204.45 Korea          
   6 Lucas Etter                    	 3194.87 USA            
   7 Sebastian Weyer                	 3179.34 Germany        
   8 Patrick Ponce                  	 3154.13 USA            
   9 Mats Valk                      	 3116.90 Netherlands    
  10 Kevin Gerhardt                 	 3060.47 Germany        
  11 Zibo Xu (徐子博)                  	 3057.62 China          
  12 Alexandre Carlier              	 3048.59 France         
  13 Ciarán Beahan                  	 3010.86 Ireland        
  14 Michał Rzewuski                	 3003.45 Poland         
  15 Phillip Lewicki                	 3003.08 USA            
  16 Co

StringIndexError: StringIndexError("Nikita Kovalchuk (Нікіта Ковальчук)", 30)